In [4]:
import pandas as pd
import torch
from torch.utils.data import Dataset

from torchvision.transforms import Compose, Lambda
from pytorchvideo.data.encoded_video import EncodedVideo

from pytorchvideo.transforms import (
    UniformTemporalSubsample,
    ShortSideScale,
    Normalize,
    UniformCropVideo
)

from pytorchvideo.data.clip_sampling import (
    UniformClipSampler,
)

In [13]:
import pandas as pd
from pytorchvideo.data import LabeledVideoDataset
from torch.utils.data import Dataset

class VideoTextDataset(Dataset):
    def __init__(self, video_csv_file, text_csv_file, labels=None, video_transform=None, text_transform=None):
        """
        Initializes the dataset with paths to videos and associated text.
        """
        # self.video_df = pd.read_csv(video_csv_file)
        self.text_df = pd.read_csv(text_csv_file)
        # self.labels_df = pd.read_csv(labels)
        # self.data_df = pd.read_csv(csv_file)
        # video_df = self.data_df[['video_id', 'label']].copy()
        # text_df = self.data_df[['text', 'label']].copy()

        self.video_dataset = LabeledVideoDataset(
            labeled_video_paths=video_csv_file,
            clip_sampler=UniformClipSampler(clip_duration=2.0),
            transform=video_transform,
            decode_audio=False
        )

        self.text_transform = text_transform

    def __len__(self):
        return len(self.video_dataset)

    def __getitem__(self, idx):

        video_data = self.video_dataset[idx]
        # text = self.data_df.iloc[idx]['text']
        text = self.text_df.iloc[idx]['text']

        if self.text_transform:
            text = self.text_transform(text)

        return {'video': video_data['video'], 'label': self.text_df['label'], 'text': text}

video_transform = Compose([
    # Example video transformations
    UniformTemporalSubsample(8),
    Lambda(lambda x: x / 255.0),
    Normalize((0.45, 0.45, 0.45), (0.225, 0.225, 0.225)),
    ShortSideScale(size=256),
    UniformCropVideo(size=256)
])

text_transform = Compose([
    # Example text transformations
    Lambda(lambda x: x.lower()),
])


In [14]:
dataset = VideoTextDataset(
    video_csv_file='../train_annotations.csv',
    text_csv_file='../text_train.csv',
    video_transform=video_transform
    # text_transform=text_transform
)

In [17]:
data = dataset[0]

NotImplementedError: Subclasses of Dataset should implement __getitem__.